
# Read/Write streamline files

## Overview

DIPY_ can read and write many different file formats. In this example
we give a short introduction on how to use it for loading or saving
streamlines. The new stateful tractogram class was made to reduce errors
caused by spatial transformation and complex file format convention.

Read `faq`


In [15]:
import os

import nibabel as nib
import numpy as np
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import load_tractogram, save_tractogram
from dipy.io.utils import (create_nifti_header, get_reference_info,
                           is_header_compatible)
from dipy.tracking.streamline import select_random_set_of_streamlines
from dipy.tracking.utils import density_map

from dipy.data.fetcher import (fetch_file_formats,
                               get_file_formats)
from dipy.io.image import load_nifti_data, load_nifti, save_nifti
from dipy.io.streamline import save_trk, save_vtk, load_vtk


First fetch the dataset that contains 5 tractography file of 5 file formats:

- cc_m_sub.trk
- laf_m_sub.tck
- lpt_m_sub.fib
- raf_m_sub.vtk
- rpt_m_sub.dpy

And their reference anatomy, common to all 5 files:

- template0.nii.gz



In [16]:
# Enables/disables interactive visualization
cwd = os.getcwd()
bundles_filename = cwd + '\\whole_brain.trk'
ref_anat_filename = cwd + '\\white.nii.gz'

reference_anatomy = nib.load(ref_anat_filename)

In [17]:
affine, dimensions, voxel_sizes, voxel_order = get_reference_info(
    ref_anat_filename)
print(affine)
print(dimensions)
print(voxel_sizes)
print(voxel_order)

[[  -0.7    0.     0.    90. ]
 [   0.     0.7    0.  -126. ]
 [   0.     0.     0.7  -72. ]
 [   0.     0.     0.     1. ]]
[260 311 260]
[0.7 0.7 0.7]
LAS


Load tractogram will support 5 file formats, functions like load_trk or
load_tck will simply be restricted to one file format

TRK files contain their own header (when written properly), so they
technically do not need a reference. (See how below)

``cc_trk = load_tractogram(bundles_filename[0], 'same')``



In [22]:
cc_trk = load_tractogram(bundles_filename,'same')
#lpt_sft = load_tractogram(bundles_filename, reference_anatomy,
#                          bbox_valid_check=False)

TypeError: Input reference is not one of the supported format

In [19]:
affine, dimensions, voxel_sizes, voxel_order = get_reference_info(
    bundles_filename)
print(affine)
print(dimensions)
print(voxel_sizes)
print(voxel_order)

[[ -0.7   0.    0.  181.3]
 [  0.   -0.7   0.  217. ]
 [  0.    0.    0.7   0. ]
 [  0.    0.    0.    1. ]]
[260 311 260]
[0.7 0.7 0.7]
LPS


In [21]:
#sft = StatefulTractogram(cc_trk, img, Space.VOX)
#bbox_valid_check =False
#sft.remove_invalid_streamlines()
save_vtk(cc_trk, "whole_brain.vtk")

AttributeError: LAS

These files contain invalid streamlines (negative values once in voxel space)
This is not considered a valid tractography file, but it is possible to load
it anyway.



Once loaded, no matter the original file format, the stateful tractogram is
self-contained and maintains a valid state. By requiring a reference the
tractogram's spatial transformation can be easily manipulated.

Let's save all files as TRK to visualize in TrackVis for example.
However, when loaded the lpt and rpt files contain invalid streamlines and
for particular operations/tools/functions it is safer to remove them



Some functions in DIPY require streamlines to be in voxel space so
computation can be performed on a grid (connectivity matrix, ROIs masking,
density map). The stateful tractogram class provides safe functions for such
manipulation. These functions can be called safely over and over, by knowing
in which state the tractogram is operating, and compute only necessary
transformations

No matter the state, functions such as ``save_tractogram`` or
``removing_invalid_coordinates`` can be called safely and the transformations
are handled internally when needed.



Now let's move them all to voxel space, subsample them to 100 streamlines,
compute a density map and save everything for visualisation in another
software such as Trackvis or MI-Brain.

To access volume information in a grid, the corner of the voxel must be
considered the origin in order to prevent negative values.
Any operation doing interpolation or accessing a grid must use the
function 'to_vox()' and 'to_corner()'



Replacing streamlines is possible, but if the state was modified between
operations such as this one is not recommended:
-> cc_sft.streamlines = cc_streamlines_vox

